Dataset has 2 values:

*   comment - text
*   sentiment - categorical feature with 2 classes - target

In [20]:
from google.colab import drive
import os.path
drive.mount('/content/gdrive', force_remount=True)
data_set_path = '/content/gdrive/MyDrive/ML/HW13/'

Mounted at /content/gdrive


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline


df = pd.read_csv(data_set_path + 'IMDB Dataset.csv')

In [22]:
import torch
import torchtext
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from torchtext.data.functional import to_map_style_dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset


from torch.utils.data import Dataset

class IMDBDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        return self.dataframe.iloc[index]

# Label encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['sentiment'])
df['sentiment'] = y_encoded


train_dataset, test_dataset = train_test_split(df, test_size=0.2, random_state=42)

from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

def build_vocabulary(datasets):
    for dataset in datasets:
      for text in dataset['review']:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, test_dataset]), min_freq=1, specials=["<UNK>"])
vocab.set_default_index(vocab["<UNK>"])
print("Vocabulary Len: {}".format(len(vocab)))
target_classes = ["Positive", "Negative"]

max_words = 25

def vectorize_batch(batch):
    #print(batch)
    X, Y = list(zip(*batch))
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens) < max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y) - 1 ## We have deducted 1 from target names to get

print(train_dataset)
test_loader  = DataLoader(IMDBDataset(test_dataset) , batch_size=1024, collate_fn=vectorize_batch, shuffle=True)
train_loader = DataLoader(IMDBDataset(train_dataset), batch_size=1024, collate_fn=vectorize_batch, shuffle=True)

train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

test_features, test_labels = next(iter(test_loader))
print(f"Feature batch shape: {test_features.size()}")
print(f"Labels batch shape: {test_labels.size()}")



Vocabulary Len: 147157
                                                  review  sentiment
39087  That's what I kept asking myself during the ma...          0
30893  I did not watch the entire movie. I could not ...          0
45278  A touching love story reminiscent of In the M...          1
16398  This latter-day Fulci schlocker is a totally a...          0
13653  First of all, I firmly believe that Norwegian ...          0
...                                                  ...        ...
11284  `Shadow Magic' recaptures the joy and amazemen...          1
44732  I found this movie to be quite enjoyable and f...          1
38158  Avoid this one! It is a terrible movie. So wha...          0
860    This production was quite a surprise for me. I...          1
15795  This is a decent movie. Although little bit sh...          1

[40000 rows x 2 columns]
Feature batch shape: torch.Size([1024, 25])
Labels batch shape: torch.Size([1024])
Feature batch shape: torch.Size([1024, 25])
Labels b

In [23]:
from torch import nn
from torch.nn import functional as F

embed_len = 50
hidden_dim = 50
n_layers=1

class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.rnn = nn.RNN(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(target_classes))

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings, torch.randn(n_layers, len(X_batch), hidden_dim))
        return self.linear(output[:,-1])

rnn_classifier = RNNClassifier()

rnn_classifier

for layer in rnn_classifier.children():
    print("Layer : {}".format(layer))
    print("Parameters : ")
    for param in layer.parameters():
        print(param.shape)

out = rnn_classifier(torch.randint(0, len(vocab), (1024, max_words)))

out.shape

Layer : Embedding(147157, 50)
Parameters : 
torch.Size([147157, 50])
Layer : RNN(50, 50, batch_first=True)
Parameters : 
torch.Size([50, 50])
torch.Size([50, 50])
torch.Size([50])
torch.Size([50])
Layer : Linear(in_features=50, out_features=2, bias=True)
Parameters : 
torch.Size([2, 50])
torch.Size([2])


torch.Size([1024, 2])

In [24]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            Y = Y + 1
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))


def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)
            print('===========')
            Y = Y + 1
            print(Y)
            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        CalcValLossAndAccuracy(model, loss_fn, val_loader)

In [25]:
from torch.optim import Adam

epochs = 15
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
rnn_classifier = RNNClassifier()
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate)

print(train_loader)
print(type(train_loader))
print(len(train_loader))

TrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

<class 'torch.utils.data.dataloader.DataLoader'>
40


  2%|▎         | 1/40 [00:00<00:16,  2.31it/s]

tensor([0, 1, 0,  ..., 0, 0, 1])


  5%|▌         | 2/40 [00:00<00:15,  2.38it/s]

tensor([1, 1, 0,  ..., 0, 0, 1])


...

tensor([0, 0, 1,  ..., 0, 0, 1])


 98%|█████████▊| 39/40 [00:18<00:00,  2.25it/s]

tensor([0, 1, 0,  ..., 1, 0, 0])
tensor([1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0])


100%|██████████| 40/40 [00:18<00:00,  2.13it/s]


Train Loss : 0.418
Valid Loss : 0.646
Valid Acc  : 0.695
